For scenario generation:

In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPU.")
    except RuntimeError as e:
        print(e)

memory_limit = 4096
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memory_limit)]
        )
        print(f"Memory limit set to {memory_limit} MB for GPU.")
    except RuntimeError as e:
        print(e)
        
import tsgm
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tsgm.models.monitors import VAEMonitor
from tsgm.utils import TSFeatureWiseScaler
from tsgm.models.cvae import BetaVAE


2024-08-26 14:30:42.189782: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-26 14:30:42.243195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-26 14:30:42.243234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-26 14:30:42.245133: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-26 14:30:42.256078: I tensorflow/core/platform/cpu_feature_guar

Memory growth enabled for GPU.
Memory limit set to 4096 MB for GPU.


2024-08-26 14:30:47.459916: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:31:00.0, compute capability: 8.9
2024-08-26 14:30:49.495355: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x40351030


In [6]:
def read_trajectory_data(csv_path):
    df = pd.read_csv(csv_path)
    trajectory_data = df[['latitude_degrees', 'longitude_degrees']].values
    print(np.shape(trajectory_data))
    return trajectory_data

print(np.shape())

dataset1 = read_trajectory_data("/home/junze/.jupyter/head-on/1/1.csv")
dataset2 = read_trajectory_data("/home/junze/.jupyter/head-on/1/2.csv")


architecture1 = tsgm.models.architectures.zoo["vae_conv5"](len(dataset1), 2, 2)
architecture2 = tsgm.models.architectures.zoo["vae_conv5"](len(dataset2), 2, 2)


scaler1 = TSFeatureWiseScaler()
scaler2 = TSFeatureWiseScaler()
scaled_data1 = scaler1.fit_transform(dataset1)
scaled_data2 = scaler2.fit_transform(dataset2)


(218, 2)
(218, 2)
(158, 2)
(158, 2)


2024-08-26 15:13:52.882741: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 742.56MiB (rounded to 778633216)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-08-26 15:13:52.882816: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-08-26 15:13:52.882841: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 112, Chunks in use: 112. 28.0KiB allocated for chunks. 28.0KiB in use in bin. 11.0KiB client-requested in use in bin.
2024-08-26 15:13:52.882859: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 4, Chunks in use: 4. 2.0KiB allocated for chunks. 2.0KiB in use in bin. 2.0KiB client-requested in use in bin.
2024-08-26 15:13:52.882874:

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: 

In [ ]:
def train_vae(scaled_data, encoder, decoder, epochs=3000, batch_size=32):
    vae = BetaVAE(encoder, decoder)
    vae.compile(optimizer=keras.optimizers.Adam())
    
    monitor = VAEMonitor()
    
    history = vae.fit(
        scaled_data, 
        epochs=epochs, 
        batch_size=batch_size,
        callbacks=[monitor]
    )
    
    return vae, monitor, history

vae1, monitor1, history1 = train_vae(scaled_data1, architecture1.encoder, architecture1.decoder)
vae2, monitor2, history2 = train_vae(scaled_data2, architecture2.encoder, architecture2.decoder)

In [ ]:
num_samples = 200
data_1 = vae1.generate(num_samples)
data_2 = vae2.generate(num_samples)

merged_data = np.concatenate((data_1, data_2), axis=1)

def visualize_trajectories(data1, data2, merged):
    fig, axs = plt.subplots(3, 1, figsize=(15, 20))
    
    axs[0].set_title("Trajectory 1")
    tsgm.utils.visualization.visualize_ts_lineplot(data1, num=5, ax=axs[0])
    
    axs[1].set_title("Trajectory 2")
    tsgm.utils.visualization.visualize_ts_lineplot(data2, num=5, ax=axs[1])
    
    axs[2].set_title("Merged Trajectories")
    tsgm.utils.visualization.visualize_ts_lineplot(merged, num=5, ax=axs[2])
    
    plt.tight_layout()
    plt.show()

visualize_trajectories(data_1, data_2, merged_data)

np.save('data_trajectory1.npy', data_1)
np.save('data_trajectory2.npy', data_2)
np.save('data_merged.npy', merged_data)



In [ ]:
def plot_vae_loss(his):
    """
    Plot the VAE loss over epochs.
    
    Parameters:
    history : keras.callbacks.History
        The history object returned by model.fit()
    """
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('VAE Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()
plot_vae_loss(his=history)